# Database CRUD Operations Testing

This notebook tests CRUD operations on the CCTV database table.

In [12]:
from utils.log_config import logger, log_setup

# Setup logging
log_setup("./logs/databaseTEST", "DBTEST")
logger.info("Starting database testing")

import psycopg2
from psycopg2.extras import RealDictCursor
import os
from dotenv import load_dotenv
from contextlib import contextmanager
from utils.log_config import logger
from typing import List, Tuple, Dict, Optional, Any, Union
load_dotenv('.env.local')

@contextmanager
def get_db_connection():
    conn = psycopg2.connect(
        dbname=os.getenv('DB_NAME'),
        user=os.getenv('DB_USER'),
        password=os.getenv('DB_PASSWORD'),
        host=os.getenv('DB_HOST'),
        port=os.getenv('DB_PORT')
    )
    try:
        yield conn
    finally:
        conn.close()

def execute_db_operation(query: str, operation_type: str, params: Optional[Union[Dict, List, Tuple]] = None, batch_size: int = 1000, fetch_type: str = 'tuple'):
    with get_db_connection() as conn:
        try:
            cursor_factory = RealDictCursor if fetch_type == 'dict' else None
            with conn.cursor(cursor_factory=cursor_factory) as cur:
                logger.info(f"[DATABASE-{operation_type.upper()}-QUERY] {query}")
                logger.info(f"[DATABASE-{operation_type.upper()}-PARAMS] {params}")
                match operation_type:
                    case 'fetch':
                        cur.execute(query, params)
                        return cur.fetchall()
                    case 'insert' | 'update' if params:
                        if isinstance(params, (list, tuple)) and params and isinstance(params[0], (dict, tuple, list)):
                            return _execute_batch(cur, query, params, batch_size)
                        else:
                            cur.execute(query, params)
                            affected_rows = cur.rowcount
                            conn.commit()
                            return affected_rows
                    case 'delete' if params:
                        cur.execute(query, params)
                        affected_rows = cur.rowcount
                        conn.commit()
                        return affected_rows
                    case _:
                        logger.error(f"[DATABASE] Invalid operation_type or missing parameters")
                        return None
        except Exception as e:
            conn.rollback()
            logger.error(f"[DATABASE] Error executing {operation_type} operation: {e}")
            raise

def _execute_batch(cur, query: str, params: List[Union[Dict, Tuple]], batch_size: int) -> int:
    total_affected_rows = 0
    for i in range(0, len(params), batch_size):
        batch = params[i:i+batch_size]
        cur.executemany(query, batch)
        total_affected_rows += cur.rowcount
    cur.connection.commit()
    return total_affected_rows


'''
table = 'cctv_locations_general'
columns = ('Cam_ID', 'Location')  # If you want to query for specified columns
# all_columns = ('*',)  # If you want to query for all columns
columns_to_check_condition = ('cam_id', 'location', 'status')
data_to_check_condition = (
    ('CAM001', 'CAM002', 'CAM003'),  # Tuple of CCTV IDs
    ('New York', 'Los Angeles'),     # Tuple of locations
    'active'                         # Single value for status
)

results = retrieve_data(table, columns, columns_to_check_condition, data_to_check_condition)

SELECT Cam_ID, Location 
FROM cctv_locations_general 
WHERE cam_id IN (%s, %s, %s) 
  AND location IN (%s, %s) 
  AND status = %sl
'''

def retrieve_data(
    table: str,
    columns: Tuple[str, ...],
    columns_to_check_condition: Optional[Tuple[str, ...]] = None,
    data_to_check_condition: Optional[Tuple[Any, ...]] = None
) -> Tuple[Tuple[Any, ...], ...]:
    try:
        # Construct the base query
        query = f"SELECT {', '.join(columns)} FROM {table}"
        
        # Add WHERE clause if conditions are provided
        params = ()
        if columns_to_check_condition and data_to_check_condition:
            where_clauses = []
            for col, data in zip(columns_to_check_condition, data_to_check_condition):
                # Check if data is either a tuple or a list
                if isinstance(data, (tuple, list)):
                    # Create the correct number of placeholders for IN clause
                    placeholders = ','.join(['%s' for _ in data])
                    where_clauses.append(f"{col} IN ({placeholders})")
                    # Extend params with each element of the list/tuple
                    params = params + tuple(data)
                else:
                    where_clauses.append(f"{col} = %s")
                    params = params + (data,)
            query += " WHERE " + " AND ".join(where_clauses)

        # Execute the query
        results = execute_db_operation(query, "fetch", params if params else None)
        
        logger.info(f"[DATABASE-RETRIEVE] Successfully retrieved data from {table}")
        return results

    except Exception as e:
        logger.error(f"[DATABASE-RETRIEVE] Error retrieving data from {table}: {e}")
        return tuple()
'''
table = 'cctv_locations_general'
columns = ('Cam_ID', 'Location', 'IsActive')
data_to_insert = (
    ('CAM001', 'New York', True),
    ('CAM002', 'Los Angeles', False),
    ('CAM003', 'Chicago', True),
    ('CAM004', 'Houston', True),
    ('CAM005', 'Phoenix', False)
)

insert_data(table, columns, data_to_insert)

INSERT INTO cctv_locations_general (Cam_ID, Location, IsActive) 
VALUES (%s, %s, %s)
'''

def insert_data(table: str, columns: Tuple[str, ...], data_to_insert: Tuple[Tuple[Any, ...], ...]) -> int:
    try:
        # Construct the base query
        placeholders = ', '.join(['%s' for _ in columns])
        query = f"INSERT INTO {table} ({', '.join(columns)}) VALUES ({placeholders})"

        # Execute the query
        rows_inserted = execute_db_operation(query, "insert", data_to_insert)
        logger.info(f"[DATABASE-INSERT] Successfully inserted {rows_inserted} rows to {table}")
        return rows_inserted

    except Exception as e:
        logger.error(f"[DATABASE-INSERT] Error inserting data into {table}: {e}")
        return 0


'''
table = 'cctv_locations_general'
columns_to_check_condition = ('cam_id', 'location', 'status')
data_to_check_condition = (
    ('CAM001', 'CAM002', 'CAM003'),  # Tuple of CCTV IDs
    ('New York', 'Los Angeles'),     # Tuple of locations
    'active'                         # Single value for status
)

rows_deleted = delete_data(table, columns_to_check_condition, data_to_check_condition)

DELETE FROM cctv_locations_general 
WHERE cam_id IN (%s, %s, %s) 
  AND location IN (%s, %s) 
  AND status = %s
'''

def delete_data(table: str, columns_to_check_condition: Tuple[str, ...], data_to_check_condition: Tuple[Union[Tuple[Any, ...], Any], ...]) -> int:
    try:
        # Construct the base query
        query = f"DELETE FROM {table}"
        
        # Add WHERE clause
        where_clauses = []
        params = {}
        
        for column, data in zip(columns_to_check_condition, data_to_check_condition):
            if isinstance(data, tuple):
                placeholders = [f"%({column}_{i})s" for i in range(len(data))]
                where_clauses.append(f"{column} IN ({', '.join(placeholders)})")
                params.update({f"{column}_{i}": value for i, value in enumerate(data)})
            else:
                where_clauses.append(f"{column} = %({column})s")
                params[column] = data
        
        query += " WHERE " + " AND ".join(where_clauses)

        rows_deleted = execute_db_operation(query, "delete", params)
                
        logger.info(f"[DATABASE-DELETE] Successfully deleted {rows_deleted} rows from {table}")
        return rows_deleted

    except Exception as e:
        logger.error(f"[DATABASE-DELETE] Error deleting data from {table}: {e}")
        return 0


'''
# Example usage 1: Update all records
table = 'cctv_locations_preprocessing'
columns_to_update = ('is_online', 'last_checked')
data_to_update = (True, '2023-10-03 12:00:00')

results1 = update_data(table, columns_to_update, data_to_update)

UPDATE cctv_locations_preprocessing 
SET is_online = %s, last_checked = %s


# Example usage 2: with multiple columns in WHERE clause
table = 'cctv_locations_preprocessing'
columns_to_update = ('is_online', 'last_checked')
data_to_update = (True, '2023-10-03 12:00:00')
columns_to_check_condition = ('cam_id', 'location', 'status')
data_to_check_condition = (
    ('CAM001', 'CAM002', 'CAM003'),  # Tuple of CCTV IDs
    ('New York', 'Los Angeles'),     # Tuple of locations
    'active'                         # Single value for status
)

results2 = update_data(table, columns_to_update, data_to_update, columns_to_check_condition, data_to_check_condition)

UPDATE cctv_locations_preprocessing 
SET is_online = %s, last_checked = %s
WHERE cam_id = ANY(%s::text[]) 
  AND location = ANY(%s::text[]) 
  AND status = %s
'''




def update_data(
    table: str,
    columns_to_update: Tuple[str, ...],
    data_to_update: Union[Tuple[Any, ...], List[Any]],
    columns_to_check_condition: Union[Tuple[str, ...], str],  # Modified to accept single string
    data_to_check_condition: Union[Tuple[Any, ...], Any]  # Modified to accept single value
) -> Optional[int]:
    """
    Build and execute UPDATE query with dynamic conditions using PostgreSQL's ANY operator.
    """
    try:
        # Validate input parameters
        if len(columns_to_update) != len(data_to_update):
            raise ValueError("Length mismatch between columns_to_update and data_to_update")
        
        # Convert single string condition to tuple
        if isinstance(columns_to_check_condition, str):
            columns_to_check_condition = (columns_to_check_condition,)
        
        # Convert single value condition to tuple
        if not isinstance(data_to_check_condition, (list, tuple)):
            data_to_check_condition = (data_to_check_condition,)
        
        if bool(columns_to_check_condition) != bool(data_to_check_condition):
            raise ValueError("Both condition parameters must be provided together")
            
        if columns_to_check_condition and len(columns_to_check_condition) != len(data_to_check_condition):
            raise ValueError(f"Length mismatch between condition columns ({len(columns_to_check_condition)}) and values ({len(data_to_check_condition)})")

        # Build SET clause
        set_clause_parts = []
        params: List[Any] = []
        
        for col in columns_to_update:
            set_clause_parts.append(f"{col} = %s")
        params.extend(data_to_update)
        
        query = f"UPDATE {table} SET {', '.join(set_clause_parts)}"
        
        # Build WHERE clause if conditions are provided
        where_clause_parts = []
        if columns_to_check_condition and data_to_check_condition:
            for col, value in zip(columns_to_check_condition, data_to_check_condition):
                if isinstance(value, (list, tuple)):
                    # Use ANY with array for multiple values
                    if not value:  # Empty list/tuple
                        raise ValueError(f"Empty list/tuple provided for column {col}")
                    # Cast the column to text for string comparison if needed
                    where_clause_parts.append(f"{col}::text = ANY(%s::text[])")
                    params.append(list(map(str, value)))  # Convert all values to strings
                else:
                    where_clause_parts.append(f"{col} = %s")
                    params.append(value)
            
            query += " WHERE " + " AND ".join(where_clause_parts)
        
        # Check if this is a batch update
        if isinstance(data_to_update[0], (list, tuple)):
            # For batch updates, restructure the parameters
            batch_params = []
            for data_set in data_to_update:
                param_set = list(data_set)
                if columns_to_check_condition and data_to_check_condition:
                    # Handle array parameters for batch updates
                    for value in data_to_check_condition:
                        if isinstance(value, (list, tuple)):
                            param_set.append(list(map(str, value)))
                        else:
                            param_set.append(value)
                batch_params.append(tuple(param_set))
            
            rows_updated = execute_db_operation(query, 'update', batch_params)
            logger.info(f"[DATABASE-UPDATE] Successfully updated {rows_updated} records")
            return
        else:
            # For single updates
            rows_updated = execute_db_operation(query, 'update', tuple(params))
            logger.info(f"[DATABASE-UPDATE] Successfully updated {rows_updated} records")
            return
            
    except Exception as e:
        logger.error(f"[DATABASE-UPDATE] Error building update query: {str(e)}")
        raise

Directory ./logs/databaseTEST exists and is writable.
2024-10-23 18:44:47,537 INFO: [MAIN] Logging setup completed!
2024-10-23 18:44:47,537 INFO: Starting database testing


## Test Data Setup
Define test data for CCTV records

```sql
CREATE TABLE cctv_test (
    id VARCHAR(100) PRIMARY KEY,
    name VARCHAR(100) NOT NULL,
    location VARCHAR(100),
    latitude DECIMAL(9,6),
    longitude DECIMAL(9,6),
    status BOOLEAN DEFAULT true,
    created_at TIMESTAMP DEFAULT CURRENT_TIMESTAMP
);
```

In [3]:
# Test data set 1 - Tuple of Tuple
cctv_records_1 = (
    ('CAM001', 'Main Entrance Cam', 'Main Entrance', 40.712776, -74.005974, True, '2024-01-01 00:00:00'),
    ('CAM002', 'Parking North', 'North Parking Lot', 40.712900, -74.006000, True, '2024-01-01 00:00:00'),
    ('CAM003', 'Back Door Cam', 'Back Door', 40.712850, -74.005920, False, '2024-01-02 00:00:00'),
    ('CAM004', 'Side Gate East', 'East Gate', 40.712890, -74.005950, True, '2024-01-02 00:00:00'),
    ('CAM005', 'Lobby Cam', 'Main Lobby', 40.712825, -74.005990, True, '2024-01-03 00:00:00')
)

# Test data set 2 - List of Tuple
cctv_records_2 = [
    ('CAM006', 'Loading Dock', 'Warehouse Entrance', 40.712760, -74.005880, True, '2024-01-03 00:00:00'),
    ('CAM007', 'Parking South', 'South Parking Lot', 40.712700, -74.005975, False, '2024-01-04 00:00:00'),
    ('CAM008', 'Reception Area', 'Front Desk', 40.712780, -74.005960, True, '2024-01-04 00:00:00'),
    ('CAM009', 'Emergency Exit', 'West Emergency Door', 40.712840, -74.006020, True, '2024-01-05 00:00:00'),
    ('CAM010', 'Elevator Cam', 'Main Elevator', 40.712795, -74.005940, True, '2024-01-06 00:00:00')
]

# Test data set 3 - List of List
cctv_records_3 = [
    ['CAM011', 'Cafeteria Cam', 'Staff Cafeteria', 40.712755, -74.005935, True, '2024-01-01 00:00:00'],
    ['CAM012', 'Conference Cam', 'Main Conference Room', 40.712785, -74.005945, False, '2024-01-01 00:00:00'],
    ['CAM013', 'Server Room', 'IT Department', 40.712775, -74.005955, True, '2024-01-01 00:00:00'],
    ['CAM014', 'Storage Cam', 'Storage Area', 40.712765, -74.005965, True, '2024-01-01 00:00:00'],
    ['CAM015', 'Archive Room', 'Document Archive', 40.712745, -74.005975, False, '2024-01-01 00:00:00']
]

# Test data set 4 - Tuple of List
cctv_records_4 = (
    ['CAM016', 'Rooftop Cam', 'Building Roof', 40.712735, -74.005985, True, '2024-01-01 00:00:00'],
    ['CAM017', 'Garden Cam', 'Courtyard Garden', 40.712725, -74.005995, True, '2024-01-01 00:00:00'],
    ['CAM018', 'Basement Cam', 'Basement Level', 40.712715, -74.006005, False, '2024-01-01 00:00:00'],
    ['CAM019', 'Garage Cam', 'Underground Parking', 40.712705, -74.006015, True, '2024-01-01 00:00:00'],
    ['CAM020', 'Security Office', 'Security Control Room', 40.712695, -74.006025, True, '2024-01-01 00:00:00']
)


cctv_status_dict = {
    'CAM001': False,
    'CAM002': True,
    'CAM003': True,
    'CAM004': False,
    'CAM005': True,
    'CAM006': False,
    'CAM007': True,
    'CAM008': True,
    'CAM009': False,
    'CAM010': True,
    'CAM011': False,
    'CAM012': True,
    'CAM013': True,
    'CAM014': False,
    'CAM015': True,
    'CAM016': False,
    'CAM017': True,
    'CAM018': False,
    'CAM019': True,
    'CAM020': False
}

## Test 1: Insert Operations

In [13]:
# Test inserting first set of records (tuple of tuple)
try:
    insert_data('cctv_test',
                ('id', 'name', 'location', 'latitude', 'longitude', 'status', 'created_at'),
                cctv_records_1
    )
    logger.info("Successfully inserted first set of records")
except Exception as e:
    logger.error(f"Error inserting first set of records: {str(e)}")

2024-10-23 14:24:34,759 INFO: [DATABASE-INSERT] Successfully inserted 5 rows to cctv_test
2024-10-23 14:24:34,760 INFO: Successfully inserted first set of records


In [14]:
# Test inserting second set of records (list of tuple)
try:
    insert_data('cctv_test',
                ['id', 'name', 'location', 'latitude', 'longitude', 'status', 'created_at'],
                cctv_records_2
    )
    logger.info("Successfully inserted second set of records")
except Exception as e:
    logger.error(f"Error inserting second set of records: {str(e)}")

2024-10-23 14:24:36,467 INFO: [DATABASE-INSERT] Successfully inserted 5 rows to cctv_test
2024-10-23 14:24:36,467 INFO: Successfully inserted second set of records


In [15]:
# Test inserting second set of records (list of list)
try:
    insert_data('cctv_test',
                ['id', 'name', 'location', 'latitude', 'longitude', 'status', 'created_at'],
                cctv_records_3
    )
    logger.info("Successfully inserted second set of records")
except Exception as e:
    logger.error(f"Error inserting second set of records: {str(e)}")

2024-10-23 14:24:38,594 INFO: [DATABASE-INSERT] Successfully inserted 5 rows to cctv_test
2024-10-23 14:24:38,595 INFO: Successfully inserted second set of records


In [16]:
# Test inserting second set of records (tuple of list)
try:
    insert_data('cctv_test',
                ['id', 'name', 'location', 'latitude', 'longitude', 'status', 'created_at'],
                cctv_records_4
    )
    logger.info("Successfully inserted second set of records")
except Exception as e:
    logger.error(f"Error inserting second set of records: {str(e)}")

2024-10-23 14:24:40,052 INFO: [DATABASE-INSERT] Successfully inserted 5 rows to cctv_test
2024-10-23 14:24:40,052 INFO: Successfully inserted second set of records


## Test 2: Retrieve Operations

In [21]:
# Test retrieving all records with list
try:
    all_records = retrieve_data('cctv_test', ['*'])
    logger.info("All records:")
    for record in all_records:
        logger.info(record)
    logger.info(f"Successfully retrieved {len(all_records)} records")
except Exception as e:
    logger.error(f"Error retrieving records: {str(e)}")

2024-10-23 14:27:54,218 INFO: [DATABASE-RETRIEVE] Successfully retrieved data from cctv_test
2024-10-23 14:27:54,218 INFO: All records:
2024-10-23 14:27:54,219 INFO: ('CAM001', 'Main Entrance Cam', 'Main Entrance', Decimal('40.712776'), Decimal('-74.005974'), True, datetime.datetime(2024, 1, 1, 0, 0))
2024-10-23 14:27:54,219 INFO: ('CAM002', 'Parking North', 'North Parking Lot', Decimal('40.712900'), Decimal('-74.006000'), True, datetime.datetime(2024, 1, 1, 0, 0))
2024-10-23 14:27:54,220 INFO: ('CAM003', 'Back Door Cam', 'Back Door', Decimal('40.712850'), Decimal('-74.005920'), False, datetime.datetime(2024, 1, 2, 0, 0))
2024-10-23 14:27:54,220 INFO: ('CAM004', 'Side Gate East', 'East Gate', Decimal('40.712890'), Decimal('-74.005950'), True, datetime.datetime(2024, 1, 2, 0, 0))
2024-10-23 14:27:54,221 INFO: ('CAM005', 'Lobby Cam', 'Main Lobby', Decimal('40.712825'), Decimal('-74.005990'), True, datetime.datetime(2024, 1, 3, 0, 0))
2024-10-23 14:27:54,221 INFO: ('CAM006', 'Loading Dock

In [3]:
# Test retrieving specific with columns list
try:
    locations = retrieve_data('cctv_test', ['id', 'location', 'status'])
    logger.info("Locations and status:")
    for record in locations:
        logger.info(record)
    logger.info("Successfully retrieved specific columns")
except Exception as e:
    logger.error(f"Error retrieving specific columns: {str(e)}")

2024-10-23 14:30:56,293 INFO: [DATABASE-RETRIEVE] Successfully retrieved data from cctv_test
2024-10-23 14:30:56,294 INFO: 
Locations and status:
2024-10-23 14:30:56,294 INFO: ('CAM001', 'Main Entrance', True)
2024-10-23 14:30:56,295 INFO: ('CAM002', 'North Parking Lot', True)
2024-10-23 14:30:56,295 INFO: ('CAM003', 'Back Door', False)
2024-10-23 14:30:56,296 INFO: ('CAM004', 'East Gate', True)
2024-10-23 14:30:56,296 INFO: ('CAM005', 'Main Lobby', True)
2024-10-23 14:30:56,297 INFO: ('CAM006', 'Warehouse Entrance', True)
2024-10-23 14:30:56,298 INFO: ('CAM007', 'South Parking Lot', False)
2024-10-23 14:30:56,298 INFO: ('CAM008', 'Front Desk', True)
2024-10-23 14:30:56,299 INFO: ('CAM009', 'West Emergency Door', True)
2024-10-23 14:30:56,299 INFO: ('CAM010', 'Main Elevator', True)
2024-10-23 14:30:56,299 INFO: ('CAM011', 'Staff Cafeteria', True)
2024-10-23 14:30:56,300 INFO: ('CAM012', 'Main Conference Room', False)
2024-10-23 14:30:56,300 INFO: ('CAM013', 'IT Department', True)
2024-

In [25]:
# Test retrieving specific with columns list and condition
try:
    locations = retrieve_data('cctv_test', ['id', 'location', 'status'], ['status', 'id'], [False,['CAM003', 'CAM015']])
    logger.info("Locations and status:")
    for record in locations:
        logger.info(record)
    logger.info("Successfully retrieved specific columns")
except Exception as e:
    logger.error(f"Error retrieving specific columns: {str(e)}")

2024-10-23 14:59:34,110 INFO: SELECT id, location, status FROM cctv_test WHERE status = %s AND id IN (%s,%s)
2024-10-23 14:59:34,111 INFO: (False, 'CAM003', 'CAM015')
2024-10-23 14:59:34,139 INFO: [DATABASE-RETRIEVE] Successfully retrieved data from cctv_test
2024-10-23 14:59:34,139 INFO: Locations and status:
2024-10-23 14:59:34,140 INFO: ('CAM003', 'Back Door', False)
2024-10-23 14:59:34,140 INFO: ('CAM015', 'Document Archive', False)
2024-10-23 14:59:34,140 INFO: Successfully retrieved specific columns


## Test 3: Update Operations

In [42]:
try:
    update_data(
        'cctv_test',
        ('status', 'created_at'),
        (True, '2024-08-08 00:00:00'),
        ('id', 'status'),
        (('CAM001', 'CAM002', 'CAM003'),False)
    )

    verify = retrieve_data('cctv_test', ['*'], ['id'], [('CAM001', 'CAM002', 'CAM003')])
    for record in verify:
        logger.info(record)
except Exception as e:
    logger.error(f"Error updating record: {str(e)}")


2024-10-23 15:27:23,754 INFO: [DATABASE-UPDATE-DEBUG] UPDATE cctv_test SET status = %s, created_at = %s WHERE id::text = ANY(%s::text[]) AND status = %s
2024-10-23 15:27:23,755 INFO: [DATABASE-UPDATE-DEBUG] [True, '2024-08-08 00:00:00', ['CAM001', 'CAM002', 'CAM003'], False]
2024-10-23 15:27:23,788 INFO: [DATABASE-UPDATE] Successfully updated 0 records
2024-10-23 15:27:23,788 INFO: [DATABASE-RETRIEVE-DEBUG] SELECT * FROM cctv_test WHERE id IN (%s,%s,%s)
2024-10-23 15:27:23,789 INFO: [DATABASE-RETRIEVE-DEBUG] ('CAM001', 'CAM002', 'CAM003')
2024-10-23 15:27:23,818 INFO: [DATABASE-RETRIEVE] Successfully retrieved data from cctv_test
2024-10-23 15:27:23,819 INFO: ('CAM001', 'Main Entrance Cam', 'Main Entrance', Decimal('40.712776'), Decimal('-74.005974'), True, datetime.datetime(2024, 8, 8, 0, 0))
2024-10-23 15:27:23,819 INFO: ('CAM002', 'Parking North', 'North Parking Lot', Decimal('40.712900'), Decimal('-74.006000'), True, datetime.datetime(2024, 8, 8, 0, 0))
2024-10-23 15:27:23,820 INFO

In [9]:
# List of CCTV IDs with True status
active_cameras = [cam_id for cam_id, status in cctv_status_dict.items() if status]

# List of CCTV IDs with False status
inactive_cameras = [cam_id for cam_id, status in cctv_status_dict.items() if not status]

# For demonstration, let's print them:
# print("Active Cameras:", active_cameras)
# Should output something like: ['CAM002', 'CAM003', 'CAM005', 'CAM007', 'CAM008', 'CAM010', 'CAM012', 'CAM013', 'CAM015', 'CAM017', 'CAM019']

# print("Inactive Cameras:", inactive_cameras)
# Should output something like: ['CAM001', 'CAM004', 'CAM006', 'CAM009', 'CAM011', 'CAM014', 'CAM016', 'CAM018', 'CAM020']

clustered_cams_coordinate = (
    ('CAM001', True, 40.712776, -74.005974),
    ('CAM002', True, 40.712900, -74.006000),
    ('CAM003', False, 40.712850, -74.005920),
    ('CAM004', True, 40.712890, -74.005950),
    ('CAM005', False, 40.712825, -74.005990)
)

cam_ids = [coord[0] for coord in clustered_cams_coordinate]
cam_bool = [coord[1] for coord in clustered_cams_coordinate]

try:
    update_data(
        'cctv_test',
        ('status', 'created_at'),
        (True, '2024-03-09 00:00:00'),
        ('id'),
        ([coord[0] for coord in clustered_cams_coordinate],)
    )

    verify = retrieve_data('cctv_test', ['*'], ['id'], [('CAM001', 'CAM002', 'CAM003')])
    for record in verify:
        logger.info(record)
except Exception as e:
    logger.error(f"Error updating record: {str(e)}")

2024-10-23 18:35:06,600 INFO: [DATABASE-UPDATE-DEBUG] UPDATE cctv_test SET status = %s, created_at = %s WHERE id::text = ANY(%s::text[])
2024-10-23 18:35:06,601 INFO: [DATABASE-UPDATE-DEBUG] [True, '2024-03-09 00:00:00', ['CAM001', 'CAM002', 'CAM003', 'CAM004', 'CAM005']]
2024-10-23 18:35:06,631 INFO: [DATABASE-UPDATE] Successfully updated 5 records
2024-10-23 18:35:06,632 INFO: [DATABASE-RETRIEVE-DEBUG] SELECT * FROM cctv_test WHERE id IN (%s,%s,%s)
2024-10-23 18:35:06,632 INFO: [DATABASE-RETRIEVE-DEBUG] ('CAM001', 'CAM002', 'CAM003')
2024-10-23 18:35:06,660 INFO: [DATABASE-RETRIEVE] Successfully retrieved data from cctv_test
2024-10-23 18:35:06,660 INFO: ('CAM001', 'Main Entrance Cam', 'Main Entrance', Decimal('40.712776'), Decimal('-74.005974'), True, datetime.datetime(2024, 3, 9, 0, 0))
2024-10-23 18:35:06,661 INFO: ('CAM002', 'Parking North', 'North Parking Lot', Decimal('40.712900'), Decimal('-74.006000'), True, datetime.datetime(2024, 3, 9, 0, 0))
2024-10-23 18:35:06,661 INFO: (

## Test 4: Delete Operations

In [14]:
# Test deleting a record
try:
    delete_data('cctv_test',
                ('id', 'status'),
                ('CAM020', False)
                )
    
    # Verify deletion
    remaining_records = retrieve_data('cctv_test', ['id'])
    logger.info("Remaining records:")
    logger.info([record[0] for record in remaining_records])
except Exception as e:
    logger.error(f"Error deleting record: {str(e)}")

2024-10-23 18:45:54,956 INFO: [DATABASE-DELETE-QUERY] DELETE FROM cctv_test WHERE id = %(id)s AND status = %(status)s
2024-10-23 18:45:54,956 INFO: [DATABASE-DELETE-PARAMS] {'id': 'CAM020', 'status': False}
2024-10-23 18:45:54,958 INFO: [DATABASE-DELETE] Successfully deleted 0 rows from cctv_test
2024-10-23 18:45:54,987 INFO: [DATABASE-FETCH-QUERY] SELECT id FROM cctv_test
2024-10-23 18:45:54,988 INFO: [DATABASE-FETCH-PARAMS] None
2024-10-23 18:45:54,989 INFO: [DATABASE-RETRIEVE] Successfully retrieved data from cctv_test
2024-10-23 18:45:54,990 INFO: Remaining records:
2024-10-23 18:45:54,990 INFO: ['CAM006', 'CAM007', 'CAM008', 'CAM009', 'CAM011', 'CAM012', 'CAM013', 'CAM014', 'CAM015', 'CAM016', 'CAM017', 'CAM019', 'CAM020', 'CAM018', 'CAM004', 'CAM005', 'CAM001', 'CAM002', 'CAM003']


## Test 5: Complex Queries

In [ ]:
# Test retrieving inactive cameras
try:
    inactive_cameras = retrieve_data('cctv_test', ['id', 'name', 'location'], {'status': False})
    print("Inactive cameras:")
    for camera in inactive_cameras:
        print(camera)
    logger.info(f"Found {len(inactive_cameras)} inactive cameras")
except Exception as e:
    logger.error(f"Error retrieving inactive cameras: {str(e)}")

# Test retrieving cameras by date range
try:
    recent_cameras = retrieve_data(
        'cctv_test',
        ['id', 'name', 'created_at'],
        {"created_at": (">", "2024-01-03 00:00:00")}
    )
    print("\nRecently added cameras:")
    for camera in recent_cameras:
        print(camera)
    logger.info(f"Found {len(recent_cameras)} recent cameras")
except Exception as e:
    logger.error(f"Error retrieving recent cameras: {str(e)}")